# RFM | 随机特征方法 Random Feature Method

<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:0rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2023 @ Authors</i></b><br/>
        <i>作者：
            <b>
            <a href="mailto:cxr123@mail.ustc.edu.cn">池煦荣 📨 </a>
            </b>
        </i>
        <br/>
        <i>日期：2023-07-06</i><br/>
        <i>共享协议：</a>本作品采用<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议</a>进行许可。</i><br/>
        <i>快速开始：点击上方的</i> <span style="background-color:rgb(85, 91, 228); color:white; padding: 3px; border-radius: 5px;box-shadow: 2px 2px 3px rgba(0, 0, 0, 0.3); font-size:0.75rem;">开始连接</span> <i>按钮，选择 <b><u>bohrium-notebook:2023-05-31</u>镜像</b> 和任意配置机型即可开始。
    </p>
</div>

## 背景介绍
微分方程数值方法在当下主流的科学研究与工业应用中都发挥着重要作用：在科学研究领域，高维的随机控制问题本质是求解高维Bellman方程；从头算分子动力学则结合了密度泛函理论和分子动力学技术，目标也是求解描述电子状态的Kohn-Sham方程和描述离子状态的Newton方程。在工业应用领域，计算机辅助工程（CAE）系统通过计算机软件对性能进行仿真分析，证实未来工程和产品功能的可用性、性能的可靠性，从而改善产品设计或协助解决各个行业的工程问题。基于微分方程表示的物理定律构成了CAE软件的基础，包括固体力学方程、流体力学方程和电磁学方程等等。
因此，微分方程数值方法是科学计算研究领域最重要的课题之一，同时它也是工业CAE软件的关键组成部分。

在这样的大背景下，许多微分方程数值方法被提出并广泛研究。其中经典的微分方程数值方法通常具有稳定的收敛阶，但依赖于网格离散的特性让它们难以处理复杂几何上的问题；而机器学习方法在这类复杂问题上具有优势，但求解误差不可控，且相比于经典方法需要很长的求解时间。作为一种新型的微分方程数值方法，本文的主角：随机特征方法（The random feature method[<sup>5</sup>](#R5), RFM）融合了经典方法和机器学习方法的优势，是一种兼具稳定收敛性和简便性的新型微分方程数值方法。

作为一个典型应用的例子，在本文中我们将考虑Helmholtz方程的求解。在电磁场问题中，由无源区的Maxwell方程组可推导出Wave equation，再经由Fourier变换就可得到如下的Helmholtz方程

\begin{equation*}
		\left\{\begin {aligned}
		(\Delta + \lambda^2) u(\boldsymbol{x}) &= f(\boldsymbol{x}) \quad && \boldsymbol{x}\in \Omega\\
		u(\boldsymbol{x})  &= U(\boldsymbol{x}) \quad && \boldsymbol{x} \in \partial\Omega
		\end {aligned}\right.
		%\label{stokesflow}
	\end{equation*}

在正式开始之前，我们将三类方法的基本特点总结如下（<font color="Red">红色</font>代表优势、<font color="Blue">蓝色</font>代表劣势，下同），后续将通过正文中的实验或讨论进一步阐述
1. 经典数值方法：
    - 线性系统往往行列数相等（条件个数=自由度个数）
    - <font color="Red">通常具有稳定的收敛阶（在log-log误差图中误差线性下降）</font>
    - <font color="Blue">依赖网格，难以处理复杂的计算区域</font>
    - <font color="Blue">无法求解高维问题（存在维数灾难）</font>
    - <font color="Blue">无法求解反问题（不可微分框架）</font>
1. 机器学习方法：
    - 线性系统行列数可以不相等（条件个数≠自由度个数）
    - <font color="Red">不依赖网格，在处理复杂区域时有显著优势</font>
    - <font color="Red">能够求解极其高维的微分方程，甚至可以用于解算子的参数化</font>
    - <font color="Red">可微分框架，能够在同一框架下求解反问题</font>
    - <font color="Blue">需要长时训练，方程求解时间长</font>
    - <font color="Blue">非凸优化导致方程数值解的精度有限，无法系统性优化</font>
    - <font color="Blue">边界罚项中罚参数的调整困难</font>
1. 随机特征方法（RFM）：
    - 线性系统行列数可以不相等（条件个数≠自由度个数）
    - <font color="Red">具有谱精度（在semi-log误差图中误差线性下降）</font>
    - <font color="Red">不依赖网格，在处理复杂区域时有显著优势</font>
    - <font color="Red">线性最小二乘优化框架，求解精度高、效率高</font>
    - <font color="Red">可微分框架，能够在同一框架下求解反问题</font>
    - <font color="Red">边界罚项中罚参数的调整容易</font>

在本Notebook中，我们将以上述Helmholtz方程的求解为例，给出可运行的代码，在同一实验设置下（线性系统规模相等）对三类方法进行对比与讨论。


## 目录

* [1. 依赖于离散的经典数值方法：以有限差分方法（FDM）为代表](#layer0)
* [2. 基于机器学习模型的微分方程数值方法：以物理信息神经网络（PINN）为代表](#layer1)
* [3. 博采众长的随机特征方法：The random feature method （RFM）](#layer2)


阅读本文可能需要30分钟以上，边运行边消化可能需要**1-2小时**。

## 依赖于离散的经典数值方法 <a id='layer0'></a>


### 不同方程形式，不同的离散方式
经典的微分方程数值方法通常需要将数学模型离散化、得到相应的数值模型，再求解离散后的方程。但值得注意的是，离散的细节根据所求解方程的形式的不同会有所不同：
- 方程强形式是我们最熟悉的方程形式，对于一维Helmholtz方程来说，它的强形式就是下式
\begin{equation*}
		\left\{\begin {aligned}
		(\Delta + \lambda^2) u(\boldsymbol{x}) &= f(\boldsymbol{x}) \quad && \boldsymbol{x}\in \Omega\\
		u(\boldsymbol{x})  &= U(\boldsymbol{x}) \quad && \boldsymbol{x} \in \partial\Omega
		\end {aligned}\right.
		%\label{stokesflow}
	\end{equation*}
基于方程强形式的方法在离散时往往会先进行方程的离散，在网格点（grid points）或称为配点（collocation points）上施加方程$$(\Delta + \lambda^2) u(x) = f(x)\quad\rightarrow\quad (\Delta + \lambda^2) u(x_i) = f(x_i)$$
此外，有限差分方法需要额外对算子进行离散。
- 方程的弱形式是强形式通过分部积分得到的，在有限维解空间中进行近似。一维Helmholtz方程的弱形式可表为：求解 $u \in H_0^1(\Omega)$，使得
$$
- \int_{\Omega} \nabla u \cdot \nabla v \mathrm{~d} V  + \lambda^2 \int_{\Omega}  u \cdot v \mathrm{~d} V=\int_{\Omega} f v \mathrm{~d} V, \quad \forall v \in H_0^1(\Omega)
$$
因此基于弱形式的方法依赖于数值积分，需要利用被积函数在有限个离散点上的函数值来计算积分近似值。**在实际实现过程中，许多方法（有限元法、谱元法等）在计算时需要先进行网格生成（如下图），这可能占据整个计算过程的70%以上的时间；此外，在几何非常复杂的情况下甚至无法进行网格生成！**

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/17477/725632357018477f98642a991fe539e0/363598cf-5e66-42ef-96f6-6cec131bd9f2.png"  width="200" alt="二维复杂区域的有限元网格示例">

- 方程的变分形式将方程求解转化为最小化势能函数的优化问题，但同样依赖于数值积分，在此不再赘述。

不管以上这些经典数值方法基于方程的哪种形式（强形式/弱形式/变分形式），<font color="Red">它们通常都具有稳定的收敛阶和完备的收敛理论</font>。但对于计算区域几何复杂的情形，<font color="Blue">网格生成困难且耗时</font>；此外，由于维度灾难的存在，这类数值方法<font color="Blue">基本无法求解高维问题</font>。

### 基于方程强形式的有限差分方法

有限差分方法是基于大家最熟悉的方程强形式的数值方法，此外，离散思想在有限差分方法的各个步骤中都有所体现，甚至其最终计算结果也是在离散点上的输出。因此我们以有限差分方法为例来介绍经典方法是如何进行离散的
1. 方程的离散：如上所述，基于强形式的有限差分方法的方程离散通过在网格点（配点）上施加方程来实现，即
$$(\Delta + \lambda^2) u(x_i) = f(x_i), \quad n=1,\cdots,N-1$$
1. 微分算子的离散：可以看到，在上面的离散方程中，Laplace算子 $\Delta$ 作用下的 $\Delta  u(x_i)$ 在求解函数值 $u^h_i = u(x_i)$ 时并不是可以直接得到的，因此考虑对Laplace算子用如下中心差分的离散格式来取代
$$\Delta  u(x_i) = \frac{u^h_{i+1} - 2 u^h_{i} + u^h_{i-1}}{h^2}=f^h_{i}$$
1. 边界条件的离散：对于边界条件 $u(x)  = U(x)$，它的离散是容易处理的，我们只需在边界的网格点上计算
$$u^h_0 = u(x_0)=U(x_0), \quad u^h_{N}=u(x_N)=U(x_N)$$
因此端点上的函数值我们可以直接得到，不需要纳入线性系统的求解过程。
1. 通过上面的三个步骤，我们实际上完成了矩阵装配、获得了待求解的线性系统
\begin{equation*}
		\frac{1}{h^2}\left[\begin{array}{ccccc}
			h^2\lambda^2-2 & 1 & & & \\
			1 & h^2\lambda^2-2 & 1 & & \\
			& \cdots & \cdots & \cdots & \\
			& & 1 & h^2\lambda^2-2 & 1 \\
			& & & 1 & h^2\lambda^2-2
		\end{array}\right]\left[\begin{array}{c}
			u_1^h \\
			u_2^h \\
			\cdots \\
			u_{N-2}^h \\
			u_{N-1}^h
		\end{array}\right]=
		\left[\begin{array}{l}
			f_1^h+u_0^h / h^2 \\
			f_2^h \\
			\cdots \\
			f_{N-2}^h \\
			f_{N-1}^h+u_N^h / h^2
		\end{array}\right]
	\end{equation*}
求解上述三对角线性系统 $Au=f$，我们就能获得所有配点 $x_i$ 上的函数值 $u^h_i = u(x_i), n=0,\cdots,N$。


有了上述的步骤拆解，下一小节中我们将给出有限差分方法的代码实现。

### 基于Python的有限差分方法代码实现

由于上一节已对有限差分法的步骤进行了详尽的描述，以下代码实现中我们将直接进行线性系统 $Au=f$ 的装配。

我们首先在计算域 $[0,8]$ 上定义真解函数，它由频率不同的 $\sin$, $\cos$ 函数的乘积生成

In [1]:
import numpy as np
import math
import time
from numpy.linalg import solve

# analytical solution parameters
AA = 1
aa = 2.0*np.pi
bb = 3.0*np.pi

left_boundary = 0.0
right_boundary = 8.0

lamb = 4
def u(x):
    return AA * np.sin(bb * (x + 0.05)) * np.cos(aa * (x + 0.05)) + 2.0

def d2u_dx2(x):
    return -AA*(aa*aa+bb*bb)*np.sin(bb*(x+0.05))*np.cos(aa*(x+0.05))\
           -2.0*AA*aa*bb*np.cos(bb*(x+0.05))*np.sin(aa*(x+0.05))

def f(x):
    return(d2u_dx2(x) + lamb*u(x))

vanal_u = np.vectorize(u)
vanal_f = np.vectorize(f)

然后我们就可以直接进行线性系统 $Au=f$ 的装配，并返回矩阵 $A$ 和向量 $f$

In [2]:
# calculate the matrix A,f in linear equations system 'Au=f'
def cal_matrix(N, points):
    A = np.zeros((N+1,N+1),dtype=np.float64)
    dx = (right_boundary - left_boundary)/N
    for i in range(1,N):
        A[i,i-1] = 1/(dx**2)
        A[i,i] = lamb - 2/(dx**2)
        A[i,i+1] = 1/(dx**2)
    A[0,0] = 1.0
    A[N,N] = 1.0
    f = np.zeros((N+1,1),dtype=np.float64)
    f[0,0] = u(points[0])
    f[1:N] = vanal_f(points[1:N]).reshape((-1,1))
    f[N] = u(points[-1])
    return(A,f)

最后，我们定义测试函数和绘图函数，并在线性系统规模分别为 $100,200,400,800,1600,3200$ 时进行测试，以体现有限差分法的收敛阶（在log-log误差图中误差线性下降）

In [3]:
# calculate the l^{inf}-norm and l^{2}-norm error for u
def test(points, u):
    true_values = vanal_u(points)
    numerical_values = u
    epsilon = true_values - numerical_values
    epsilon = np.maximum(epsilon, -epsilon)
    error_inf = epsilon.max()
    error_l2 = math.sqrt(8*sum(epsilon*epsilon)/len(epsilon))
    print('L_infty=',error_inf,'L_2=',error_l2)
    return(error_l2)

def error_plot(multi_Errors):
    import matplotlib.pyplot as plt
    plt.figure(figsize=[7, 5])
    plt.tick_params(labelsize=10)
    font2 = {
    'weight' : 'normal',
    'size'   : 22,
    }
    plt.xlabel('Degrees of freedom',font2)
    plt.ylabel('$L_2$ absolute error',font2)
    plt.xscale('log')
    plt.yscale('log')
    Label = ['FDM','PINN','RFM']
    for i in range(len(multi_Errors)):
        Error = multi_Errors[i]
        plt.plot(Error[:,0], Error[:,1], \
                 lw=1.5, ls='-', clip_on=False,\
                 marker='o', markersize=10, \
                 label = Label[i],\
                 markerfacecolor='none',\
                 markeredgewidth=1.5)
    plt.legend()
    plt.title("Comparison of accuracy on 1D Helmholtz equation")
    plt.show()

def time_plot(multi_Errors):
    import matplotlib.pyplot as plt
    plt.figure(figsize=[7, 5])
    plt.tick_params(labelsize=10)
    font2 = {
    'weight' : 'normal',
    'size'   : 22,
    }
    plt.xlabel('Degrees of freedom',font2)
    plt.ylabel('Solving time',font2)
    Label = ['FDM','PINN','RFM']
    for i in range(len(multi_Errors)):
        Error = multi_Errors[i]
        plt.plot(Error[:,0], Error[:,2], \
                 lw=1.5, ls='-', clip_on=False,\
                 marker='o', markersize=10, \
                 label = Label[i],\
                 markerfacecolor='none',\
                 markeredgewidth=1.5)
    plt.legend()
    plt.title("Comparison of efficiency on 1D Helmholtz equation")
    plt.show()
    
def main(N):
    time_begin = time.time()
    points = np.linspace(0, 8.0, N+1)
    A,f = cal_matrix(N,points)
    u = solve(A,f).reshape((-1))
    error = test(points, u)
    time_end = time.time()
    return(error, time_end - time_begin)

if __name__ == '__main__':
    FDM_Error = np.zeros([5,3])
    for i in range(5):
        N = int(100 * 2**i)
        print('***********************')
        print("N = M =",N)
        FDM_Error[i,0] = N
        FDM_Error[i,1], FDM_Error[i,2] = main(N)
    
    error_plot([FDM_Error])

***********************
N = M = 100
L_infty= 0.4038518006880172 L_2= 0.6818204170645585
***********************
N = M = 200
L_infty= 0.10009073936912527 L_2= 0.16727260903655303
***********************
N = M = 400
L_infty= 0.024869758483598403 L_2= 0.04167428236543972
***********************
N = M = 800
L_infty= 0.006221760357883799 L_2= 0.010413267709307231
***********************
N = M = 1600
L_infty= 0.0015548590687854968 L_2= 0.0026033988607706963


### 小结（经典数值方法）
1. 在上述log-log误差曲线中，我们可以很明显地看出线性趋势，<font color="Red">这与有限差分方法的理论分析一致、说明它有着非常严格的收敛阶</font>；
1. 另一方面，虽然在一维Helmholtz问题中并不明显，但有限差分方法依赖于规则网格点离散的特性，使得它<font color="Blue">在求解复杂几何问题时会遇到困难</font>，需要引入虚拟点（如下图）并设计特殊的差分格式来解决这一问题；类似的，基于网格的有限元、谱元法等经典数值方法需要在整个求解域内生成网格，同样难以处理复杂几何上的问题；
1. 由于有限差分方法在计算时需要将计算域离散，若对每个维度都进行 $N$ 等分的离散，则在 $d$ 维问题中，规则网格的自由度达到 $N^d$ 的指数增长，即我们常说的维度灾难问题，这也导致这类经典数值方法<font color="Blue">基本无法求解高维问题</font>。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/17477/eb548f21a3ab44db8ca4038f52b95f3d/096e10a4-c3f3-4329-ae4f-1143230c20c2.png"  width="200" alt="二维圆弧上有限差分方法虚拟点示例">


## 基于机器学习模型的微分方程数值方法 <a id='layer1'></a>

基于机器学习的微分方程数值方法是近年来的新兴方法，与经典数值方法相比，机器学习方法的长处在于求解高维的PDE和控制问题，甚至可以被用来进行PDE解算子的参数化。但机器学习方法误差具有不可控性；且模型需要长时训练，因此方程求解所需时间往往较长。

### 如何借助机器学习模型求解微分方程
从以上的例子我们可以看出，经典的微分方程数值方法从大框架上看有三个主要的构成
- 数值离散：配点（强形式）、数值积分（弱形式、变分形式）
- 逼近空间：全局/局部基函数的线性组合
- 优化问题：线性方程组求解

注意到，在机器学习方法中，有完全类似的概念
- 数值离散：配点、数值积分构造损失函数
- 逼近空间：神经网络模型
- 优化问题：非线性优化，可使用随机梯度下降（SGD）

因此引入机器学习模型来求解微分方程的想法是非常自然的。且同样自然地，针对不同的方程形式，有对应的机器学习方法来求解：

- 强形式: Deep Galerkin Method[<sup>1</sup>](#R1) (DGM) 和 Physics-Informed Neural Networks[<sup>2</sup>](#R2) (PINN) 在配点上定义损失函数，
$$
L(u)=\left\|(\Delta + \lambda^2) u - f\right\|_{2,\Omega}^2+\lambda\|u-U\|_{2, \partial \Omega}^2
$$
其中方程的强形式用最小二乘形式给出，再加上带罚参数的边界条件罚项来组合损失函数。
- 弱形式: 基于方程弱形式构造的机器学习方法中，最著名的是 Weak Adversarial Network[<sup>3</sup>](#R3) (WAN) 方法，它通过借鉴生成对抗网络（GAN）思想，将弱形式的方程求解转化为$min-max$问题的优化求解。具体来说，首先定义算子 $\mathcal{A}$ 和 $\mathcal{B}$，将弱形式方程重写为
\begin{equation*}
	\left\{\begin{array}{l}
		\langle\mathcal{A}[u], v\rangle \triangleq \int_{\Omega}\left(\nabla u \cdot \nabla v+u \cdot  v-f \cdot v\right) \mathrm{d} x=0 \\
		\mathcal{B}[u]=0, \quad \text { on } \partial \Omega
	\end{array}\right.
	\end{equation*}
算子 $\mathcal{A}$ 的模可定义为 $\|\mathcal{A}[u]\|_{o p} \triangleq \max \left\{\langle\mathcal{A}[u], v\rangle /\|v\|_2 \mid v \in H_0^1, v \neq 0\right\}$，
则原方程项等价于如下优化问题形式
$$
	\min _{u \in H^1}\|\mathcal{A}[u]\|_{\text {op }}^2 \Longleftrightarrow \min _{u \in H^1} \max _{v \in H_0^1}|\langle\mathcal{A}[u], v\rangle|^2 /\|v\|_2^2
$$
对右端取$log$，得到方程项的损失函数
$$L_{\text {int }}(\theta, \eta) \triangleq \log \left|\left\langle\mathcal{A}\left[u_\theta\right], v_\eta\right\rangle\right|^2-\log \left\|v_\eta\right\|_2^2$$
再类似PINN方法，构造边界条件罚项
$$
	L_{\text {bdry }}(\theta) \triangleq\left(1 / N_b\right) \cdot \sum_{j=1}^{N_b}\left|u_\theta\left(x_b^{(j)}\right)-g\left(x_b^{(j)}\right)\right|^2 \\
$$
最终的优化问题就可写为
$$
\min _\theta \max _\eta L(\theta, \eta), \quad \text { where } L(\theta, \eta) \triangleq L_{\text {int }}(\theta, \eta)+\alpha L_{\text {bdry }}(\theta)
$$
- 变分形式: Deep Ritz Method[<sup>4</sup>](#R4) (DRM) 方法的损失函数则是将势能函数加上边界条件罚项得到的，数值计算、优化求解和上述方法都是一致的，在此不赘述。

### 最经典的神经网络方法——PINN[<sup>2</sup>](#R2)

作为神经网络方法的典型例子，我们将介绍该领域中最广为人知的PINN方法。

为与下文的RFM进行对比，PINN的网络结构我们使用有单隐层的全连接神经网络，网络模型与权重初始化方式定义如下

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
torch.set_default_dtype(torch.float64)

R_m = 1.0
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.uniform_(m.weight, a = -R_m, b = R_m)
        nn.init.uniform_(m.bias, a = -R_m, b = R_m)

# network definition
class Network(nn.Module):
    def __init__(self, d, M):
        super(Network, self).__init__()
        self.fc_layer = nn.Sequential(nn.Linear(d, M, bias=True),nn.Tanh())
        self.output_layer = nn.Linear(M, 1, bias = False)
        
    def forward(self, x):
        h = self.fc_layer(x)
        out = self.output_layer(h)
        return out

有了模型，下一步自然是构造损失函数。正如上文中所提到的，PINN的损失函数由如下
$$
L(u)=\left\|(\Delta + \lambda^2) u - f\right\|_{2,\Omega}^2+\lambda\|u-U\|_{2, \partial \Omega}^2
$$
在实际实现中，我们同样要对方程进行离散，在配点上配置损失函数（优化问题），展开写为如下形式：
$$
L(u)=\sum_{i=1}^{N-1}((\Delta + \lambda^2) u(x_i) - f(x_i))^2+\lambda_l(u(x_0)-U(x_0))^2+\lambda_r(u(x_N)-U(x_N))^2
$$

In [5]:
# loss definition
class Loss():
    def __init__(self, net, left_boundary, right_boundary):
        self.net = net
        self.left_boundary = left_boundary
        self.right_boundary = right_boundary

    def sample(self, N):
        x = torch.tensor(np.float64(np.linspace(self.left_boundary, self.right_boundary, N+1)[1:N]),requires_grad=True).reshape(-1,1)
        x_boundary_left = torch.ones([1],requires_grad=True) * self.left_boundary
        x_boundary_right = torch.ones([1],requires_grad=True) * self.right_boundary
        return x, x_boundary_left, x_boundary_right

    def loss_func(self, N):
        # helmholtz equation
        x, x_boundary_left, x_boundary_right = self.sample(N)
        x = Variable(x, requires_grad=True)
        y = self.net(x)
        dx = torch.autograd.grad(y, x, grad_outputs=torch.ones_like(self.net(x)), create_graph=True)[0].reshape(-1, 1)
        dxx = torch.autograd.grad(dx, x, grad_outputs=torch.ones_like(dx), create_graph=True)[0].reshape(-1, 1)
        
        f = -AA*(aa*aa+bb*bb)*torch.sin(bb*(x+0.05))*torch.cos(aa*(x+0.05))\
           -2.0*AA*aa*bb*torch.cos(bb*(x+0.05))*torch.sin(aa*(x+0.05))\
           -lamb * (AA * torch.sin(bb * (x + 0.05)) * torch.cos(aa * (x + 0.05)) + 2.0)
        diff_error = (dxx - lamb*self.net(x) - f.reshape(-1, 1))**2
        
        # boundary condition
        bd_left_error = (self.net(x_boundary_left) - (AA * torch.sin(bb * (x_boundary_left + 0.05)) * torch.cos(aa * (x_boundary_left + 0.05)) + 2.0)) ** 2
        bd_right_error = (self.net(x_boundary_right) - (AA * torch.sin(bb * (x_boundary_right + 0.05)) * torch.cos(aa * (x_boundary_right + 0.05)) + 2.0)) ** 2
        
        return torch.mean(diff_error + bd_left_error + bd_right_error)

最后的优化部分，由于是非凸优化问题，我们直接使用通用的Adam优化算法求解，训练过程可以定义如下

In [6]:
# training process
class Train():
    def __init__(self, net, loss, N):
        self.errors = []
        self.N = N
        self.net = net
        self.model = loss

    def train(self, epoch, lr):
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.net.parameters()), lr)
        avg_loss = 0
        for e in range(epoch):
            optimizer.zero_grad()
            loss = self.model.loss_func(self.N)
            avg_loss = avg_loss + float(loss.item())
            loss.backward()
            optimizer.step()
            if e % 50 == 49:
                loss = avg_loss/50
                #print("Epoch {} - lr {} -  loss: {}".format(e, lr, loss))
                avg_loss = 0
                error = self.model.loss_func(self.N)
                self.errors.append(error.detach())

最后，我们定义了测试过程，并同样在线性系统规模分别为 $100,200,400,800,1600,3200$ 时进行测试，以便与有限差分方法做对比

In [7]:
# test process
def Test(net, N):
    # PINN testing process
    points = np.linspace(left_boundary, right_boundary, N+1, dtype=np.float64)[:N]
    true_value = u(points).reshape([-1,1])
    numerical_value = net(torch.tensor(points,requires_grad=False).reshape([-1,1])).detach().cpu().numpy()
    epsilon = true_value - numerical_value
    epsilon = np.abs(epsilon)
    error_inf = epsilon.max()
    error_l2 = math.sqrt(8*sum(epsilon*epsilon)/len(epsilon))
    print('L_infty=',error_inf,'L_2=',error_l2)
    return(error_l2)

if __name__=="__main__":
    PINN_Error = np.zeros([5,3])
    for i in range(5):
        time_begin = time.time()
        N = int(100 * 2**i) # number of collocation points
        M = int(100 * 2**i) # number of basis
        
        # PINN model define and initialization
        net = Network(1, M)
        net.fc_layer = net.fc_layer.apply(weights_init)
        
        # PINN training process
        FREEZE = False
        if FREEZE == True:
            net.fc_layer.requires_grad_(False)
            loss = Loss(net, left_boundary, right_boundary)
            train = Train(net, loss, N)
            train.train(epoch=10**2, lr=0.001)
            train.train(epoch=4*10**2, lr=0.0001)
        else:
            loss = Loss(net, left_boundary, right_boundary)
            train = Train(net, loss, N)
            train.train(epoch=5*10**3, lr=0.1)
            train.train(epoch=10**4, lr=0.01)
            train.train(epoch=10**4, lr=0.001)
            train.train(epoch=2*10**4, lr=0.0001)
        
        # PINN testing process
        print('***********************')
        print("N = M =",N)
        PINN_Error[i,0] = N
        PINN_Error[i,1] = Test(net, N)
        time_end = time.time()
        PINN_Error[i,2] = time_end - time_begin

    error_plot([FDM_Error, PINN_Error])
    time_plot([FDM_Error, PINN_Error])

***********************
N = M = 100
L_infty= 0.10487853885758636 L_2= 0.0594229909371562
***********************
N = M = 200
L_infty= 0.01510564043982976 L_2= 0.00966925733986969
***********************
N = M = 400
L_infty= 0.03488366954326194 L_2= 0.023427789973332842
***********************
N = M = 800
L_infty= 0.007590964691803981 L_2= 0.006726670710635236
***********************
N = M = 1600
L_infty= 0.01069138024637617 L_2= 0.00775340961692466


### 小结（机器学习方法）
1. 从误差对比图可以看出，通过简单网络架构实现的<font color="Blue">PINN在任何自由度下结果都可接受，但误差并不随模型自由度的增加而下降</font>，这可能是由于模型与训练参数并没有精细调整，但更主要的原因是PINN的训练过程是非凸非线性的优化过程，<font color="Blue">容易陷入局部最优</font>；且<font color="Blue">罚参数的调整较为依赖经验</font>，并不直接；
1. 从时间对比图可以看出，不仅误差不随自由度的增大而下降，<font color="Blue">PINN的训练时间也远远超过经典有限差分方法的求解时间</font>，这也是本节所介绍的机器学习方法的通病；
1. 尽管在一维Helmholtz问题上没有体现，但由于机器学习方法无需事先生成网格，只需在计算域中采点进行训练，因此<font color="Red">不管在复杂几何的问题还是高维问题中，机器学习方法都更占优势</font>；
1. 由于机器学习方法通常是以可微分的映射表示，因此<font color="Red">可以在同一框架下统一地处理正反问题</font>，也<font color="Red">可以用作算子的参数化</font>；
1. 回归到PINN框架下，一些文献[<sup>6</sup>](#R6)说明其原始模型中所构造的损失函数并不适用于所有方程，构造其他损失函数和训练算法后结果明显变好；
1. 总的来说，基于机器学习的微分方程数值方法还存在较多的改进空间。


## 随机特征方法：The random feature method[<sup>5</sup>](#R5) <a id='layer2'></a>

从上文对经典方法和机器学习方法的介绍，我们可以看出：经典方法有稳定的收敛阶，但难以处理复杂几何或高维问题；机器学习方法能处理复杂几何和高维问题，但求解时间长、求解精度有限。因此，如何结合经典方法与机器学习方法的优势，提出同时兼具精度和处理复杂几何能力的方法在科学计算领域一直是重要的问题。

作为一种新型的微分方程数值方法，本文的主角：随机特征方法（The random feature method, RFM）融合了经典方法和机器学习方法的优势，能够实现谱精度（semi-log误差图中线性），其无网格的特性也让它能够容易地处理复杂几何上的问题，是一种兼具收敛性和简便性的新型微分方程数值方法。
     
类似前文两类方法的主要构成，RFM的主要构成同样分成如下三部分
- 数值离散：基于方程强形式，在配点上构造线性系统
- 逼近空间：局部随机特征函数，并用单位分解技术将其组合
- 优化问题：线性最小二乘问题，容易求解，罚参数易于调整

下文我们将以对应的三小节逐步构建RFM的框架。

### 逼近空间

#### **随机特征函数**
什么是随机特征函数（random feature function, RFF）？随机特征函数就是特征向量随机生成的函数。对于机器学习方法来说，就是网络权值随机初始化的步骤。因此从机器学习的框架下看，RFM就是利用 $M$ 个定义在 $\Omega$ 上的网络基函数 $\{\phi_m\}$ 的线性组合来表示数值解
	\begin{equation*}\label{eqn:rfm}
		u_M(\boldsymbol{x}) = \sum_{m=1}^M u_m \phi_m(\boldsymbol{x})
	\end{equation*}
	$$
	\phi_m(\boldsymbol{x}) = \sigma(\boldsymbol{k}_m \cdot\boldsymbol{x} + b_m)
	$$
其中 $\boldsymbol{k}_m, b_m$ 就是随机生成后固定的内层参数，而 $\sigma$ 是非线性激活函数，为方程求解提供非线性的部分。

#### **局部RFF与单位分解**
上述随机特征函数是全局定义的，但微分方程的解常常有小尺度的局部变化，因此RFM考虑在多个局部构造局部随机特征函数，再用单位分解（partition of unity）技术将它们组合。

具体来说，首先取定单位分解函数的中心点 $\{\boldsymbol{x}_n\}_{n=1}^{M_p}\subset\Omega$，在这 $M_p$ 个局部构造仿射变换
\begin{equation}
	\tilde{\boldsymbol{x}}=\frac{1}{\boldsymbol{r}_{n}}(\boldsymbol{x}-\boldsymbol{x}_{n}), \quad n=1,\cdots, M_p,
	\end{equation}
这个仿射变换将 $[x_{n1}-r_{n1},x_{n1}+r_{n1}]\times \cdots \times [x_{nd}-r_{nd},x_{nd}+r_{nd}]$ 的小局部映射到统一的区间 $[-1,1]^{d}$，以便实现局部特征的拟合。而单位分解函数的构造依赖于这个仿射变换，常取为
\begin{equation*}
	\psi_{n}^{a}(x)=\mathbb{I}_{-1 \leq \tilde{x} < 1},
	\label{psi1}
	\end{equation*}
	或
	\begin{equation*}
	\psi_{n}^{b}(x) =\mathbb{I}_{\left[-\frac{5}{4},-\frac{3}{4}\right]}(\tilde{x}) \frac{1+\sin (2 \pi \tilde{x})}{2}+\mathbb{I}_{\left[-\frac{3}{4}, \frac{3}{4}\right]}(\tilde{x})+\mathbb{I}_{\left[\frac{3}{4}, \frac{5}{4}\right]}(\tilde{x}) \frac{1-\sin (2 \pi \tilde{x})}{2}
	\end{equation*}
    
<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/17477/bea71b3b2ef34af9828b21e52ae60d12/63fc00d5-b4d1-4089-8faa-3659a46a4dce.png"  width="300" alt="两类一维PoU函数示例">
紧接着，在每个局部定义 $J_n$ 个随机特征函数
	\begin{equation}\label{eqn:basis0}
	\phi_{nj}(\boldsymbol{x}) = \sigma(\boldsymbol{k}_{nj} \cdot \tilde{\boldsymbol{x}} + b_{nj}), \quad j=1, \cdots, J_n,
	\end{equation}

则最终数值解就是将局部随机特征函数通过单位分解函数组合起来
\begin{equation*}
		u_M(\boldsymbol{x})=\sum_{n=1}^{M_p} \psi_n (\boldsymbol{x})   \sum_{j=1}^{J_n }u_{nj} \phi_{nj} (\boldsymbol{x})
		\label{representation2}
		\end{equation*}
        
*以下实验代码使用$\psi_{n}^{a}(x)$作为单位分解函数进行演示*

In [8]:
# computational domain
X_min = 0.0
X_max = 8.0

# random initialization for parameters
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.uniform_(m.weight, a = -R_m, b = R_m)
        nn.init.uniform_(m.bias, a = -R_m, b = R_m)

# random feature basis when using \psi^{a} as PoU function
class RFM_rep_a(nn.Module):
    def __init__(self, d, J_n, x_max, x_min):
        super(RFM_rep_a, self).__init__()
        self.d = d
        self.J_n = J_n
        self.r = (x_max - x_min) / 2.0
        self.x_c = (x_max + x_min)/2
        self.phi = nn.Sequential(nn.Linear(self.d, self.J_n, bias=True),nn.Tanh())

    def forward(self,x):
        x = (x - self.x_c) / self.r
        x = self.phi(x)
        return x

# random feature basis when using \psi^{b} as PoU function
class RFM_rep_b(nn.Module):
    def __init__(self, d, J_n, x_max, x_min):
        super(RFM_rep_b, self).__init__()
        self.d = d
        self.J_n = J_n
        self.n = x_min/(X_max-X_min) * M_p
        self.r = (x_max - x_min) / 2.0
        self.x_0 = (x_max + x_min)/2
        self.phi = nn.Sequential(nn.Linear(self.d, self.J_n, bias=True),nn.Tanh())

    def forward(self,x):
        d = (x - self.x_0) / self.r
        psi = ((d <= -3/4) & (d > -5/4)) * (1+torch.sin(2*np.pi*d))/2 + ((d <= 3/4) & (d > -3/4)) * 1.0 + ((d <= 5/4) & (d > 3/4)) * (1-torch.sin(2*np.pi*d))/2
        y = self.phi(d)
        if self.n == 0:
            psi = ((d <= 3/4) & (d > -1)) * 1.0 + ((d <= 5/4) & (d > 3/4)) * (1-torch.sin(2*np.pi*d))/2
        elif self.n == M_p-1:
            psi = ((d <= -3/4) & (d > -5/4)) * (1+torch.sin(2*np.pi*d))/2 + ((d <= 1) & (d > -3/4)) * 1.0
        else:
            psi = ((d <= -3/4) & (d > -5/4)) * (1+torch.sin(2*np.pi*d))/2 + ((d <= 3/4) & (d > -3/4)) * 1.0 + ((d <= 5/4) & (d > 3/4)) * (1-torch.sin(2*np.pi*d))/2
        return(psi*y)

# predefine the random feature functions in each PoU region
def pre_define(M_p,J_n):
    models = []
    for n in range(M_p):
        x_min = (X_max-X_min)/M_p * n + X_min
        x_max = (X_max-X_min)/M_p * (n+1) + X_min
        model = RFM_rep_a(d = 1, J_n = J_n, x_min = x_min, x_max = x_max)
        model = model.apply(weights_init)
        model = model.double()
        for param in model.parameters():
            param.requires_grad = False
        models.append(model)
    return(models)

### 数值离散：线性系统构造

在RFM中，损失函数同样在配点上进行计算，和PINN的损失函数构造完全一致，以最小二乘形式计算方程项，并加上边界的罚项
\begin{equation}
	Loss = \sum_{\boldsymbol{x}_{i} \in C_{I}}\sum_{k=1}^{K_I}\lambda_{Ii}^{k}\|\mathcal{L}^{k}\boldsymbol{u}(\boldsymbol{x}_{i})-\boldsymbol{f}^{k}(\boldsymbol{x}_{i})\|_{l^{2}}^{2}+ \sum_{\boldsymbol{x}_{j} \in C_{B}}\sum_{\ell=1}^{K_B}\lambda_{Bj}^{\ell}\|\mathcal{B}^{\ell}\boldsymbol{u}(\boldsymbol{x}_{j})-\boldsymbol{g}^{\ell}(\boldsymbol{x}_{j})\|_{l^{2}}^{2}.
	\label{loss2}
	\end{equation}

虽然损失函数形式上看完全类似，但RFM与PINN构造的优化问题则完全不同，RFM求解的是内层参数固定、仅优化最外层线性参数的线性优化问题。从经典数值算法的角度来看，RFM构造的是线性最小二乘问题，该问题可以表为线性系统 $Au=f$ 的形式。因此罚参数 $\lambda_i$ 的调整可以直接基于矩阵 $A$ 的信息，对于每个配点、每个方程，<font color="Red">RFM允许他们有不同的罚参数</font>。

对应于一维Helmholtz方程，RFM的损失函数可以写为
$$
L(u)=\sum_{i=1}^{N-1}\lambda_i((\Delta + \lambda^2) u(x_i) - f(x_i))^2+\lambda_0(u(x_0)-U(x_0))^2+\lambda_N(u(x_N)-U(x_N))^2
$$

在实际代码实现中，我们通过下述配点上的等式构造线性系统 $Au=f$ 来等价地求解该优化问题
\begin{equation*}
		\left\{\begin {aligned}
		(\Delta + \lambda^2) u(x_i) &= f(x_i), \quad &&n=1,\cdots,N-1\\
		u(x_{i})  &= U(x_{i}) \quad && n=0,N
		\end {aligned}\right.
		%\label{stokesflow}
	\end{equation*}

*需要注意的是，在单位分解函数的连续性不满足方程解所需的连续性时，需要额外加入一组连续性条件。例如：在使用$\psi^{a}$求解二阶方程时，需要在不同单位分解计算区域的边缘加入零阶和一阶连续性条件*

In [9]:
# Assembling the matrix A,f in linear system 'Au=f'
def assemble_matrix(models,points,M_p,J_n,Q,lamb):
    A_I = np.zeros([M_p*Q, M_p*J_n]) # PDE term
    A_B = np.zeros([2, M_p*J_n]) # boundary condition
    A_C_0 = np.zeros([M_p-1, M_p*J_n]) # 0-order smoothness condition
    A_C_1 = np.zeros([M_p-1, M_p*J_n]) # 1-order smoothness condition
    f = np.zeros([M_p*Q + 2*(M_p - 1) + 2, 1])
    
    for n in range(M_p):
        # forward and grad
        point = torch.tensor(points[n], requires_grad=True)
        out = models[n](point)
        values = out.detach().numpy()
        value_l, value_r = values[0,:], values[-1,:]
        grad1 = []
        grad2 = []
        for i in range(J_n):
            g1 = torch.autograd.grad(outputs=out[:,i], inputs=point,
                                  grad_outputs=torch.ones_like(out[:,i]),
                                  create_graph = True, retain_graph = True)[0]
            grad1.append(g1.squeeze().detach().numpy())
            
            g2 = torch.autograd.grad(outputs=g1[:,0], inputs=point,
                                  grad_outputs=torch.ones_like(out[:,i]),
                                  create_graph = False, retain_graph = True)[0]
            grad2.append(g2.squeeze().detach().numpy())
        grad1 = np.array(grad1).T
        grad2 = np.array(grad2).T
        grad_l = grad1[0,:]
        grad_r = grad1[-1,:]
        Lu = grad2 - lamb * values
        
        # Lu = f condition
        A_I[n*Q:(n + 1)*Q, n*J_n:(n + 1)*J_n] = Lu[:Q,:]
        f[n*Q:(n + 1)*Q, :] = F(points[n], lamb).reshape([-1,1])[:Q]
        
        # boundary conditions
        if n == 0:
            A_B[0, :J_n] = value_l
        if n == M_p-1:
            A_B[1, -J_n:] = value_r
        
        # smoothness conditions
        if M_p > 1:
            if n == 0 :
                A_C_0[0, :J_n] = -value_r
                A_C_1[0, :J_n] = -grad_r
            elif n == M_p - 1:
                A_C_0[M_p - 2, -J_n:] = value_l
                A_C_1[M_p - 2, -J_n:] = grad_l
            else:
                A_C_0[n-1,n*J_n:(n + 1)*J_n] = value_l
                A_C_1[n-1,n*J_n:(n + 1)*J_n] = grad_l
                A_C_0[n,n*J_n:(n + 1)*J_n] = -value_r
                A_C_1[n,n*J_n:(n + 1)*J_n] = -grad_r
    if M_p > 1:
        A = np.concatenate((A_I,A_B,A_C_0,A_C_1),axis=0)
    else:
        A = np.concatenate((A_I,A_B),axis=0)
    
    # boundary conditions
    f[M_p*Q,:] = u(0.)
    f[M_p*Q+1,:] = u(8.)
    
    return(A,f)

### 优化问题

由于一些实际问题中物理参数变化较大，因此RFM基于矩阵 $A$ 的信息（将矩阵 $A$ 每行最大值rescaling到同一规模），使用如下自动调参方案来平衡损失函数中各项的贡献，其中罚参数设为
	\begin{align*}
			&\lambda_{Ii}^{k} = \frac{c}{\underset{1\leq n\leq M_p}{\max}\underset{1\leq j'\leq J_n}{\max}\underset{1\leq k'\leq K_I}{\max}|\mathcal{L}^{k} (\phi^{k'}_{nj'}(\boldsymbol{x}_{i})\psi_{n}(\boldsymbol{x}_{i}))|}\quad \boldsymbol{x}_{i} \in C_{I}, \; k = 1,\cdots, K_I\\
			&\lambda_{Bj}^{\ell} = \frac{c}{\underset{1\leq n\leq M_p}{\max}\underset{1\leq j'\leq J_n}{\max}\underset{1\leq \ell'\leq K_I}{\max}|\mathcal{B}^{\ell} (\phi^{\ell'}_{nj'}(\boldsymbol{x}_{j})\psi_{n}(\boldsymbol{x}_{j}))|}\quad \boldsymbol{x}_{j} \in C_{B}, \; \ell = 1,\cdots, K_B 
		\end{align*}

对应于一维Helmholtz方程，这些罚参数可以写为
	\begin{align*}
			&\lambda_{i} = \frac{100}{\underset{1\leq n\leq M_p}{\max}\underset{1\leq j'\leq J_n}{\max}|(\Delta + \lambda^2) (\phi_{nj'}(x_{i})\psi_{n}(x_{i}))|}\quad i = 1,\cdots, N-1\\
			&\lambda_{i} = \frac{100}{\underset{1\leq n\leq M_p}{\max}\underset{1\leq j'\leq J_n}{\max}|\phi_{nj'}(x_{i})\psi_{n}(x_{i})|}\quad i = 0,N
		\end{align*}

In [10]:
from scipy.linalg import lstsq
def main(M_p, J_n, Q, lamb):
    # prepare collocation points
    time_begin = time.time()
    points = []
    for n in range(M_p):
        x_min = (X_max-X_min)/M_p * n + X_min
        x_max = (X_max-X_min)/M_p * (n+1) + X_min
        points.append(np.linspace(x_min, x_max, Q+1).reshape([-1,1]))
    
    # prepare models
    models = pre_define(M_p,J_n)
    
    # matrix define (Au=f)
    A,f = assemble_matrix(models, points, M_p, J_n, Q, lamb)
    print('***********************')
    print('Matrix shape: N=%s,M=%s'%(A.shape))
    # rescaling
    c = 100.0
    for i in range(len(A)):
        ratio = c/A[i,:].max()
        A[i,:] = A[i,:]*ratio
        f[i] = f[i]*ratio
    
    # solve
    w = lstsq(A,f)[0]
    
    # test
    error = test(models,M_p,J_n,Q,w)
    
    time_end = time.time()
    return(error, time_end - time_begin)

### 实验1：与有限差分方法 / PINN方法的对比实验
经过上面的准备，我们已搭建完RFM的整体框架。同样地，我们定义RFM的测试过程，实现 $[0,8]$ 区间上的一维Helmholtz方程求解

In [11]:
# analytical solution parameters
AA = 1
aa = 2.0*np.pi
bb = 3.0*np.pi
lamb = 4

vanal_u = np.vectorize(u)
vanal_d2u_dx2 = np.vectorize(d2u_dx2)

def F(points, lamb):
    return(vanal_d2u_dx2(points) - lamb * vanal_u(points))

import math
import matplotlib.pyplot as plt
# calculate the l^{infty}-norm and l^{2}-norm error for u
def test(models,M_p,J_n,Q,w,plot = False):
    epsilon = []
    true_values = []
    numerical_values = []
    test_Q = 2*Q
    for n in range(M_p):
        points = torch.tensor(np.linspace((X_max-X_min)/M_p * n + X_min, (X_max-X_min)/M_p * (n+1) + X_min, test_Q+1),requires_grad=False).reshape([-1,1])
        out = models[n](points)
        values = out.detach().numpy()
        true_value = vanal_u(points.numpy()).reshape([-1,1])
        numerical_value = np.dot(values, w[n*J_n:(n+1)*J_n,:])
        true_values.extend(true_value)
        numerical_values.extend(numerical_value)
        epsilon.extend(true_value - numerical_value)
    true_values = np.array(true_values)
    numerical_values = np.array(numerical_values)
    epsilon = np.array(epsilon)
    epsilon = np.maximum(epsilon, -epsilon)
    print('R_m=%s,M_p=%s,J_n=%s,Q=%s'%(R_m,M_p,J_n,Q))
    print('L_infty error =',epsilon.max(),', L_2 error =',math.sqrt(8*sum(epsilon*epsilon)/len(epsilon)))
    x = [((X_max - X_min)/M_p)*i / test_Q  for i in range(M_p*(test_Q+1))]
    return(math.sqrt((X_max-X_min)*sum(epsilon*epsilon)/len(epsilon)))

if __name__ == '__main__':
    lamb = 4
    R_m = 3
    J_n = 50 # the number of basis functions per PoU region
    Q = 50 # the number of collocation pointss per PoU region
    RFM_Error = np.zeros([5,3])
    for i in range(5): # the number of PoU regions
        M_p = 2*(2**i)
        RFM_Error[i,0] = int(M_p * J_n)
        RFM_Error[i,1], RFM_Error[i,2] = main(M_p,J_n,Q,lamb)
    error_plot([FDM_Error, PINN_Error, RFM_Error])
    time_plot([FDM_Error, PINN_Error, RFM_Error])

***********************
Matrix shape: N=104,M=100
R_m=3,M_p=2,J_n=50,Q=50
L_infty error = 39.73441344537901 , L_2 error = 30.231211492127343
***********************
Matrix shape: N=208,M=200
R_m=3,M_p=4,J_n=50,Q=50
L_infty error = 0.002640724565257102 , L_2 error = 0.0019328785522688856
***********************
Matrix shape: N=416,M=400
R_m=3,M_p=8,J_n=50,Q=50
L_infty error = 5.530362181538351e-07 , L_2 error = 5.391978823035383e-07
***********************
Matrix shape: N=832,M=800
R_m=3,M_p=16,J_n=50,Q=50
L_infty error = 4.257653940520356e-08 , L_2 error = 3.06632546585445e-08
***********************
Matrix shape: N=1664,M=1600
R_m=3,M_p=32,J_n=50,Q=50
L_infty error = 1.0577603237749145e-09 , L_2 error = 1.241129484555056e-09


**由上面的结果，我们可以发现：在相同的矩阵规模下，RFM的结果远好于有限差分方法与PINN方法的结果。**

### 实验2：RFM谱精度实验
为展现RFM的谱精度，我们取较小的矩阵规模，再次运行并绘制semi-log误差图如下

*我们重新定义了部分函数，删除输出以便多次实验取均值*

In [12]:
def test(models,M_p,J_n,Q,w,plot = False):
    epsilon = []
    true_values = []
    numerical_values = []
    test_Q = 2*Q
    for n in range(M_p):
        points = torch.tensor(np.linspace((X_max-X_min)/M_p * n + X_min, (X_max-X_min)/M_p * (n+1) + X_min, test_Q+1),requires_grad=False).reshape([-1,1])
        out = models[n](points)
        values = out.detach().numpy()
        true_value = vanal_u(points.numpy()).reshape([-1,1])
        numerical_value = np.dot(values, w[n*J_n:(n+1)*J_n,:])
        true_values.extend(true_value)
        numerical_values.extend(numerical_value)
        epsilon.extend(true_value - numerical_value)
    true_values = np.array(true_values)
    numerical_values = np.array(numerical_values)
    epsilon = np.array(epsilon)
    epsilon = np.maximum(epsilon, -epsilon)
    #print('R_m=%s,M_p=%s,J_n=%s,Q=%s'%(R_m,M_p,J_n,Q))
    #print('L_infty error =',epsilon.max(),', L_2 error =',math.sqrt(8*sum(epsilon*epsilon)/len(epsilon)))
    x = [((X_max - X_min)/M_p)*i / test_Q  for i in range(M_p*(test_Q+1))]
    return(math.sqrt((X_max-X_min)*sum(epsilon*epsilon)/len(epsilon)))

def main(M_p, J_n, Q, lamb):
    # prepare collocation points
    points = []
    for n in range(M_p):
        x_min = (X_max-X_min)/M_p * n + X_min
        x_max = (X_max-X_min)/M_p * (n+1) + X_min
        points.append(np.linspace(x_min, x_max, Q+1).reshape([-1,1]))
    
    # prepare models
    models = pre_define(M_p,J_n)
    
    # matrix define (Au=f)
    A,f = assemble_matrix(models, points, M_p, J_n, Q, lamb)
    #print('***********************')
    #print('Matrix shape: N=%s,M=%s'%(A.shape))
    # rescaling
    c = 100.0
    for i in range(len(A)):
        ratio = c/A[i,:].max()
        A[i,:] = A[i,:]*ratio
        f[i] = f[i]*ratio
    
    # solve
    w = lstsq(A,f)[0]
    
    # test
    return(A.shape[0],test(models,M_p,J_n,Q,w))

def error_plot_semi_log(Error):
    import matplotlib.pyplot as plt
    plt.figure(figsize=[7, 5])
    plt.tick_params(labelsize=10)
    font2 = {
    'weight' : 'normal',
    'size'   : 22,
    }
    plt.xlabel('Degrees of freedom',font2)
    plt.ylabel('$L_2$ absolute errors',font2)
    plt.yscale('log')
    plt.plot(Error[:,0], Error[:,1], \
             color='black', lw=1.5, ls='-', clip_on=False,\
             marker='o', markersize=10, \
             markerfacecolor='none',\
             markeredgecolor='black',markeredgewidth=1.5)
    plt.title("Spectral accuracy of RFM")
    plt.show()

if __name__ == '__main__':
    lamb = 4
    R_m = 3
    M_p = 4 # the number of PoU regions
    J_n = 50 # the number of basis functions per PoU region
    
    Error = np.zeros([8,2])
    for i in range(8):
        Q = int(10 * (i+1)) # the number of collocation pointss per PoU region
        m = []
        for o in range(40): # 40 experiments for mean
            N, error = main(M_p,J_n,Q,lamb)
            m.append(error)
        Error[i,0] = N
        Error[i,1] = np.array(m).mean()
    error_plot_semi_log(Error)
    

**由上面的结果，我们可以发现：在固定基函数数量的情况下，由小到大逐步增加配点个数，RFM的精度有明显的谱精度（semi-log误差图近似为直线）。**

### 小结（RFM）
1. 由实验1的误差对比图，我们可以看出在自由度一致的情况下，<font color="Red">RFM的结果显著优于有限差分方法与PINN方法，精度更优且下降更快</font>；
1. 实验2的semi-log误差图中，线性下降的误差曲线说明<font color="Red">RFM实际上具有谱精度</font>；
1. 由实验1的时间对比图，我们可以看出由于在线性最小二乘框架下进行计算，<font color="Red">RFM的求解（优化）时间也远远少于PINN方法</font>，与有限差分方法约在同一量级；
1. 与机器学习方法类似，RFM无需事先生成网格，只需在计算域中采点进行训练，因此<font color="Red">RFM也能容易地处理复杂几何和高维问题</font>；
1. RFM所构造的逼近空间同样是可微分映射，因此<font color="Red">可以在同一框架下统一地处理正反问题</font>；
1. 相比于PINN的训练过程，RFM的优化问题是线性最小二乘问题，所以<font color="Red">罚参数的调整非常方便</font>；
1. 在实验2中，我们可以发现在线性最小二乘的求解框架中，线性系统的行列数无需相等，这使得在构建线性系统时有更好的灵活性。

## 总结与讨论
本文通过在一维Helmholtz方程上的实验对比，说明RFM是一种兼具稳定收敛性和简便性的新型微分方程数值方法。总的来说，RFM避免了经典数值算法中的网格生成步骤，更容易处理复杂几何和高维问题；通过引入随机特征函数，在线性最小二乘优化框架下求解优化问题，避免了机器学习方法中的非凸优化，使得其精度和效率都显著更好；可以说RFM为PDE数值求解提供了一个新的视角。
除了文中的简单例子，RFM在高维的固体力学、流体力学、电磁场等实际应用中同样证明了其精度与效率[<sup>5</sup>](#R5)。

在机器学习席卷了学术界和工业界的今天，神经网络模型的拟合能力使其革新了许多领域的范式，但对于高精度要求的微分方程求解来说，目前发展的机器学习方法还不够，甚至带来了一些新的问题。本文抛砖引玉，通过介绍RFM如何结合经典数值方法和机器学习方法的优势，设计出了一种新的计算范式，希望能给读者一些启发。在我们看来，如何将强大的机器学习方法，真正在微分方程求解问题、在科学计算领域用对、用好，真正发挥出它的长处，还是一个尚待探索的问题。

## 参考文献
<div><a name="R1"></a>
[1] J. A. Sirignano and K. Spiliopoulos, DGM: A deep learning algorithm for solving partial differential equations, Journal of Computational Physics, 375 (2018), pp. 1339–1364
</div>
<div><a name="R2"></a>
[2] M. Raissi, P. Perdikaris, and G. E. Karniadakis, Physics-informed neural networks: A deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations, Journal of Computational Physics, 378 (2019), pp. 686–707.
</div>
<div><a name="R3"></a>
[3] Y. Zang, G. Bao, X. Ye, and H. Zhou, Weak adversarial networks for highdimensional partial differential equations, Journal of Computational Physics, 411
(2020), p. 109409.
</div>
<div><a name="R4"></a>
[4] W. E and B. Yu, The Deep Ritz Method: A Deep Learning-Based Numerical Algorithm for Solving Variational Problems, Communications in Mathematics and Statistics, 6 (2018), pp. 1–12.
</div>
<div><a name="R5"></a>
[5] J. Chen, X. Chi, W. E, and Z. Yang, Bridging traditional and machine learningbased algorithms for solving pdes: The random feature method, Journal of Machine
Learning, 1 (2022), pp. 268–298.
</div>
<div><a name="R6"></a>
[6] C. Wang, S. Li, D. He, and L. Wang, Is $ L^ 2$ Physics Informed Loss Always Suitable for Training Physics Informed Neural Network?. Advances in Neural Information Processing Systems, 35 (2022), pp. 8278-8290.
</div>